In [1]:
import os
import sys
import collections
sys.path.append('..')
from utils.Generate import Generate
from utils.Evaluation_indicators import Evaluation_indicators
from configs.Generate_config import *
from peft import PeftModel
import transformers
import torch
import json
import numpy as np
from accelerate.utils import release_memory
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd

2024-07-07 21:06:22.088159: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-07 21:06:22.133668: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-07 21:06:22.133691: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-07 21:06:22.135318: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-07 21:06:22.143413: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
train_path = './Train/Output/2024-06-26(Llama3)'
different_parameter_string = [train_path + '/' +i for i in os.listdir(train_path)]

save_string = ['./Generate/Output/Lora/' + train_path[-16:] + '/' +i for i in os.listdir(train_path)]

print('模型路径')
for i in different_parameter_string:
    print(i)
print('-------------------')
print('保存路径')
for i in save_string:
    print(i)
print('-------------------')

### 推理

In [ ]:
test = Generate(**CONFIG)
# 读取可用模型序列
#base_model = 'meta-llama/Llama-2-13b-hf'
base_model = 'meta-llama/Meta-Llama-3-8B'
use_tokenizer = test.load_tokenizer(base_model)

In [ ]:
PROMPTER_first_question = test.load_template(prompt_template_path = "alpaca")
#PROMPTER_other_questions = test.load_template(prompt_template_path = "alpaca")
predict_data = test.load_data("LLM2 code\\new_version\\Dataset\\Answer_test(4)_fixed.json")
#predict_data = test.load_data("../Train/Dataset/Fast_test/Test_Set.json")
pre_data = test.Preprocess()
b_data,o_data = test.map_dataset(test.data_preprocess,add_eos_token=True)

In [ ]:
loaded_model = test.load_model()
generation_config = test.Parameter_setting()

In [ ]:
for i in different_parameter_string:
    print(i)
    index = i.split('/')[-1]
    
    model_lora = PeftModel.from_pretrained(
        loaded_model,
        i,
        torch_dtype=torch.float32,        
    )
    
    for j in ['o_data']:
        p_d = test.predict_circulate(eval(j),is_shut_down=False,shut_down_index=1)
        save_file = p_d.remove_columns(column_names=['input_ids',])
        save_path = save_string[different_parameter_string.index(i)]
        test.save_results(save_file,output_dir=save_path)

### 评估

In [2]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [6]:
baseline_string = ['./Generate/Output/chat-gpt/06-16']
Evluator = Evaluation_indicators()
evaluete_string = ['/root/autodl-tmp/new_version/Generate/Output/Lora/ut/2024-06-26(Llama3)/checkpoint-400']
print('评估路径')
evaluete_string

评估路径


['/root/autodl-tmp/new_version/Generate/Output/Lora/ut/2024-06-26(Llama3)/checkpoint-400']

In [7]:
for i in evaluete_string:
    for j in ['/Only/']:
        path = i + j
        json_path = path + os.listdir(path)[0]
        print(json_path)
        evluate_data = Evluator.load_data(json_path)
        result = Evluator.run_evaluate(evluate_data, ['bleu','rouge','meteor','bert_score'])  
        # if you can't use connect to the huggingface,use local metric.You can see local metric in the './Model/Metrics/metrics'
        Evluator.save_json(result, path + 'result.json')
        # 将result保存为json文件

/root/autodl-tmp/new_version/Generate/Output/Lora/ut/2024-06-26(Llama3)/checkpoint-400/Only/Llama3_2345.json
Loading data: "/root/autodl-tmp/new_version/Generate/Output/Lora/ut/2024-06-26(Llama3)/checkpoint-400/Only/Llama3_2345.json"


Generating train split: 0 examples [00:00, ? examples/s]

Predict data loaded successfully
bleu loaded
rouge loaded


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


meteor loaded
bert_score loaded


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 读取评估结果

In [ ]:
# 想要读取推理的结果的话，用下面的方法
from datasets import load_dataset
kkk = load_dataset('json', data_files='/root/autodl-tmp/new_version/Generate/Output/Lora/2024-05-28/10-54/checkpoint-120/Only/result.json')
kkk['train'].to_pandas()